In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
data_folder = '/home/lantro/Documents/Algo Trading/LEAN/data/yahoo/'
ticker = 'spy' #'eurusd=x'
#df = yf.download(ticker, start='2022-03-17', interval='30m')
#df.to_csv(data_folder+ticker+'.csv')
df = pd.read_csv(f"{data_folder}{ticker}.csv", index_col=[0], parse_dates=True, skipinitialspace=True)
start_date = '2020-01-01' #Start date for data
end_date = '2022-05-01'
strategy_start = None #This will hold the date when the strategy starts trading to use in calculating returns.
    #To be fair we should note the date when our signals have enough data so we don't exclude dates for B&H return
df = df.loc[start_date:end_date]

In [ ]:
df.head()

In [ ]:
# This is my attempt to separate out the plotting from the strategy.
def MACD(df, s, l, signal):
    #Create new dataframe to hold signal values with same index as original df
    # Future: use lag parameter to offset signals when using longer s, l time periods otherwise when you try to use the 
    # combination of MACD signals for buy/sell signals, they won't line up in time, i.e. macd with shorter ema time will get signals earlier
    signal_df = pd.DataFrame(index=df.index)
    signal_df["macd"] = df.close.ewm(span=s, min_periods=1).mean() - df.close.ewm(span=l, min_periods=1).mean()
    signal_df["signal"] = signal_df.macd.ewm(span=signal, min_periods=1).mean()
    signal_df["diff"] = signal_df["macd"] - signal_df["signal"]

    # Cross points (buys / sells)
    # buy_sell_hold: 1 = buy, 0 = hold, -1 = sell
    for i in range(1, len(df)): # This is a slow way to do this. Should use apply or iterrows
        if signal_df.iloc[i-1]["diff"] < 0 and signal_df.iloc[i]["diff"] > 0: # MACD above signal after being below on previous day
            #print("{}:GOLDEN CROSS".format(df.iloc[i].name))
            df.loc[df.iloc[i].name,f's_macd_{s}_{l}_{signal}'] = 1

        elif signal_df.iloc[i-1]["diff"] > 0 and signal_df.iloc[i]["diff"] < 0: # MACD below signal after being above the previous day
            #print("{}:DEAD CROSS".format(df.iloc[i].name))
            df.loc[df.iloc[i].name,f's_macd_{s}_{l}_{signal}'] = -1
        else:
            # Don't do anything, no new signal
            #print("{}:NO CROSS".format(df.iloc[i].name))
            df.loc[df.iloc[i].name,f's_macd_{s}_{l}_{signal}'] = 0  
    return(df,signal_df)


In [ ]:
df, macd_df = MACD(df, 12, 26, 9)
#df, macd2_df = MACD(df, 15, 30, 12)

#df[df.iloc[:,-1] == 1] # All rows where last column value = 1
#df[df.iloc[:,7:] == 1] # All rows where last column value = 1
macd_df.head()
#df.head(20)
# View rows where either signal column signaled a buy 
df.loc[(df.filter(regex='^s_.*').values == 1).any(axis=1)]

In [ ]:
def trend(df, short_period, long_period, pct_over=1):
    #Calculate the required moving averages and difference between the two and return as df
    signal_df = pd.DataFrame(index=df.index)
    signal_df[f'{short_period}_EMA'] = df.close.ewm(span=short_period, min_periods=short_period).mean()
    signal_df[f'{long_period}_EMA'] = df.close.ewm(span=long_period, min_periods=long_period).mean()
    signal_df[f'{short_period}-{long_period}_diff'] = signal_df[f'{short_period}_EMA']-signal_df[f'{long_period}_EMA']
    signal_df[f's_uptrend_{short_period}-{long_period}'] = (np.where((signal_df[f'{short_period}-{long_period}_diff'] >= (signal_df[f'{long_period}_EMA']*pct_over*.01)), 1, -1))
    # Don't add to original df #df[f's_uptrend_{short_period}-{long_period}'] = signal_df[f's_uptrend_{short_period}-{long_period}']
    return signal_df

In [ ]:
test = trend(df, 20, 50,1)
#test.tail()
test.filter(regex='^s_.*')

In [ ]:
#df.drop('s_uptrend_20-50', axis=1, inplace=True)
df.tail()

In [ ]:
def bullish_ranking(df):
    # We can use some trend indicators to determine how long or short we want to be
    # Calculate some trend indicators, then sum to create a 'bullish' ranking to add to the main df.
    bullish_df = pd.DataFrame(index=df.index)
    bullish_df = bullish_df.join(trend(df, 20, 50,1).filter(regex='^s_.*')) #Just get the signal values from the df
    # Stock above 20 MA
    # 20 MA above 50 MA
    # 50 MA above 200 MA
    return bullish_df

bullish_ranking(df)
#signal_df[signal_df['bullishness']==1]
#df[df['bullishness']==1]


In [ ]:
def set_holdings(df, short=False):
    # We are using the 'all' numpy method which evaluates to True only if all items in array evaluate to True
    # Alternatively, we can use the 'any' method
    df['buy_sell'] = 0 # Create column to hold buy/sell signals for plotting 
    df['holdings'] = np.NaN # Create new 'holdings column'
    # LONG signals - set holdings
    df.loc[(df.filter(regex='^s_.*').values == 1).all(axis=1),'buy_sell'] = 1
    # SHORT signals - set holdings
    df.loc[(df.filter(regex='^s_.*').values == -1).all(axis=1),'buy_sell'] = -1
    # Set holdings
    df.loc[df['buy_sell'] == 1,'holdings'] = 1
    if short == True:
        df.loc[df['buy_sell'] == -1,'holdings'] = -1
    else:
        #df.loc[(df.filter(regex='^s_.*').values == -1).all(axis=1),'holdings'] = 0
        df.loc[df['buy_sell'] == -1,'holdings'] = -1
    # NEUTRAL signals - we will just fill forward values from rows that had signals to cells with NaN values
    df['holdings'].ffill(inplace=True)
    return(df)


set_holdings(df, short=True)
df.head(20)
#df[df['buy_sell'] == -1]
df[df['holdings'] == 1]

    

In [ ]:
def plot_MACD(df,macd_df):
    xdate = [x.date() for x in df.index]
    plt.figure(figsize=(20, 25))
    
    # plot the original closing line
    plt.subplot(311)
    plt.plot(xdate, df.close, label="close")
    plt.scatter(df[df["buy_sell"]==1].index, df[df["buy_sell"]==1]["close"], marker="^", s=100, color="b", alpha=0.9)
    plt.scatter(df[df["buy_sell"]==-1].index, df[df["buy_sell"]==-1]["close"], marker="v", s=100, color="r", alpha=0.9)
    plt.plot(df.close.ewm(span=50, min_periods=25).mean(), label='50_day_EMA')
    plt.plot(df.close.ewm(span=200, min_periods=50).mean(), label='200_day_EMA')
    
    
    plt.xlim(xdate[0], xdate[-1])
    plt.legend()
    plt.grid()
    
    # plot MACD and signal
    plt.subplot(312)
    plt.title("MACD")
    plt.plot(xdate, macd_df.macd, label="macd")
    plt.plot(xdate, macd_df.signal, label="signal")
    plt.xlim(xdate[0], xdate[-1])
    plt.legend()
    plt.grid(True)

    #Plot Buy signals
    plt.scatter(df[df["buy_sell"]==1].index, macd_df[df["buy_sell"]==1]["macd"], marker="o", s=100, color="b", alpha=0.9)
    #Plot Sell signals
    plt.scatter(df[df["buy_sell"]==-1].index, macd_df[df["buy_sell"]==-1]["macd"], marker="o", s=100, color="r", alpha=0.9)
    # Plot Buy / Sell Signals
    plt.subplot(313)
    plt.plot(xdate, df.holdings, label="Holdings")


In [ ]:
plot_MACD(df,macd_df)


In [ ]:
def calculate_returns(df,price_column): #
    # Calculate strategy returns (note date of first trade to use for buy & hold return calc)
    # Calculate buy & hold returns (make sure you adjust dates to account for warmup period)
    strategy_start = df[df['holdings'] == 1].index[0] #Set the trading start date to first buy for strategy
    df = df.loc[strategy_start:,:].copy() #Truncate df to start when first strategy buy trade is made
    #df_daily_returns = df.loc[strategy_start:,price_column].pct_change()
    df['bh_daily_returns'] = df[price_column].pct_change()
    df['strategy_daily_returns'] = df['bh_daily_returns'] * df['holdings']
    df = df.iloc[1:,:] #Skip 1st column containing NaN value
    #print(df_daily_returns.tail())
    
    # Calculate the cumulative daily returns
    df['bh_cum_daily_returns'] = (1 + df['bh_daily_returns']).cumprod() - 1
    df['strategy_cum_daily_returns'] = (1 + df['strategy_daily_returns']).cumprod() - 1
    #print(df_cum_daily_returns.tail())
    bh_cum_return_entire_period = df['bh_cum_daily_returns'].iloc[-1]#.tail(1)
    bh_total_return = round(bh_cum_return_entire_period * 100,2)
    print(f'Total B&H Return:  {bh_total_return}%')
    strategy_cum_return_entire_period = df['strategy_cum_daily_returns'].iloc[-1]#.tail(1)
    strategy_total_return = round(strategy_cum_return_entire_period * 100,2)
    print(f'Total Strategy Return:  {strategy_total_return}%')
    
    print(f'Start Price {dt.datetime.date(df.iloc[0].name)}:  {round(df.close.iloc[0],2)}')
    print(f'End Price {dt.datetime.date(df.iloc[-1].name)}:  {round(df.close.iloc[-1],2)}')
    
    

#strategy_start = ('2020-01-08')
calculate_returns(df,'close')